# Análise comparativa de modelos

In [23]:
from IPython.display import display, Markdown
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

## 1. Obtenção de Dados

In [11]:
df = pd.read_csv("../data/raw/data.csv")
df_dict = pd.read_csv("../data/external/dictionary.csv")
df_dict

,variavel,descricao,tipo,subtipo
0,gender,Indica o genêro do paciente,qualitativa,nominal
1,age,Indica a idade do paciente,quantitativa,discreta
2,hypertension,Indica se o paciente possui hipertensão,quantitativa,discreta
3,heart_disease,Indica se o pacinete possui doença no coração,quantitativa,discreta
4,smoking_history,Indica informações sobre o historico de fumant...,qualitativa,nominal
5,bmi,Indica o valor do IMC(Indice de Massa Corporal...,quantitativa,contínua
6,HbA1c_level,Indica o nivel de açucar presente no sangue do...,quantitativa,contínua
7,blood_glucose_level,Indica o nivel de glicose presente no sangue d...,quantitativa,discreta
8,diabetes,Indica se o paciente possui diabetes,quantitativa,discreta


## 2. Preparação de dados

In [21]:
# Definindo a variável alvo e as colunas nominais, contínuas e discretas
target_column = 'diabetes'  # A variável alvo

# Filtrando as colunas nominais e contínuas
nominal_columns = (
    df_dict
    .query("subtipo == 'nominal' and variavel != @target_column")
    .variavel
    .to_list()
)

continuous_columns = (
    df_dict
    .query("subtipo == 'continua'")
    .variavel
    .to_list()
)

# Filtrando as colunas discretas
discrete_columns = (
    df_dict
    .query("subtipo == 'discreta'and variavel != @target_column")
    .variavel
    .to_list()
)

# Separando as features (X) da variável alvo (y)
X = df.drop(columns=[target_column], axis=1)
y = df[target_column]

### 2.1 Pré-processamento

In [24]:
# Pré-processador para variáveis nominais
nominal_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='most_frequent')),  # Tratamento de dados faltantes
    ('encoding', OneHotEncoder(sparse_output=False, drop='first')),  # Codificação de variáveis nominais
])

# Pré-processador para variáveis contínuas
continuous_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='mean')),  # Tratamento de dados faltantes
    ('normalization', StandardScaler())  # Normalização de dados contínuos
])

# Pré-processador para variáveis discretas
discrete_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='most_frequent')),  # Tratamento de dados faltantes
    ('normalization', MinMaxScaler())  # Normalização de variáveis discretas
])

# Combinando os pré-processadores usando ColumnTransformer
preprocessor = ColumnTransformer([
    ('nominal', nominal_preprocessor, nominal_columns),
    ('continuous', continuous_preprocessor, continuous_columns),
    ('discrete', discrete_preprocessor, discrete_columns)
])

# Aplicando o pré-processamento ao conjunto de dados
X_preprocessed = preprocessor.fit_transform(X)

# Verificando os resultados do pré-processamento
print("X_preprocessed shape:", X_preprocessed.shape)
print("X_preprocessed sample:", X_preprocessed[:5])

X_preprocessed shape: (100000, 11)
X_preprocessed sample: [[0.         0.         0.         0.         0.         1.
  0.         1.         0.         1.         0.27272727]
 [0.         0.         0.         0.         0.         0.
  0.         0.67467467 0.         0.         0.        ]
 [1.         0.         0.         0.         0.         1.
  0.         0.34934935 0.         0.         0.35454545]
 [0.         0.         1.         0.         0.         0.
  0.         0.44944945 0.         0.         0.34090909]
 [1.         0.         1.         0.         0.         0.
  0.         0.94994995 1.         1.         0.34090909]]
